In [35]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

def facilities_that_charge(conn):
    query = "SELECT name FROM FACILITIES WHERE membercost > 0"
    df = pd.read_sql_query(query, conn)
    return df

def facilities_no_charge(conn):
    query = "SELECT COUNT(name) FROM FACILITIES WHERE membercost = 0"
    df = pd.read_sql_query(query, conn)
    return df

def less_than_maint(conn):
    query = "SELECT facid, name, membercost, monthlymaintenance FROM FACILITIES WHERE membercost < (monthlymaintenance/5) AND membercost <> 0"
    df = pd.read_sql_query(query, conn)
    return df

def one_and_five(conn):
    query = "SELECT * FROM FACILITIES WHERE facid IN (1, 5)"
    df = pd.read_sql_query(query, conn)
    return df

def cheap_or_expensive(conn):
    query = "SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS cost_label FROM FACILITIES"
    df = pd.read_sql_query(query, conn)
    return df

def last_members(conn):
    query = "SELECT firstname, surname FROM MEMBERS WHERE joindate = (SELECT max(joindate) FROM MEMBERS)"
    df = pd.read_sql_query(query, conn)
    return df

def tennis_members(conn):
    query = "SELECT DISTINCT f.name AS court_name, \
                m.surname || ' ' || m.firstname AS member_name \
                FROM BOOKINGS b \
                LEFT JOIN MEMBERS m ON b.memid = m.memid \
                LEFT JOIN FACILITIES f ON b.facid = f.facid \
                WHERE f.name LIKE 'Tennis%' \
                ORDER BY member_name"
    df = pd.read_sql_query(query, conn)
    return df

def more_than_30(conn):
    query = "SELECT b.bookid, f.name AS facility_name, m.surname || ' ' || m.firstname AS member_name, CASE \
                WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS cost FROM Bookings b \
                LEFT JOIN Facilities f ON b.facid = f.facid \
                LEFT JOIN Members m ON b.memid = m.memid WHERE b.starttime LIKE '2012-09-14%' AND CASE WHEN b.memid = 0 THEN f.guestcost * b.slots \
                ELSE f.membercost * b.slots END > 30 ORDER BY cost"
    df = pd.read_sql_query(query, conn)
    return df

def more_than_30_subquery(conn):
    query = "SELECT sub.bookid, m.surname || ' ' || m.firstname as member_name, sub.cost FROM (SELECT b.bookid, b.memid, \
                CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS cost FROM Bookings b \
                LEFT JOIN Facilities f ON b.facid = f.facid WHERE b.starttime LIKE '2012-09-14%') sub LEFT JOIN Members m ON sub.memid = m.memid \
                WHERE sub.cost > 30 ORDER BY sub.cost"
    df = pd.read_sql_query(query, conn)
    return df

def less_than_thousand(conn):
    query = "SELECT f.name AS facility_name, SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS total_revenue \
                FROM Bookings b LEFT JOIN Facilities f ON b.facid = f.facid GROUP BY f.name HAVING total_revenue < 1000 ORDER BY total_revenue"
    df = pd.read_sql_query(query, conn)
    return df

def member_report(conn):
    query = "SELECT m1.surname || ', ' || m1.firstname AS member_name, m2.surname || ', ' || m2.firstname AS recommended_by FROM Members m1 \
                LEFT JOIN Members m2 ON m1.recommendedby = m2.memid ORDER BY m1.surname, m1.firstname"
    df = pd.read_sql_query(query, conn)
    return df

def facilities_usage_member(conn):
    query = "SELECT f.name AS facility_name, m.surname || ', ' || m.firstname AS member_name, SUM(b.slots) AS usage FROM Bookings b JOIN \
                Members m ON b.memid = m.memid JOIN Facilities f ON b.facid = f.facid WHERE b.memid != 0 GROUP BY f.name, m.surname, m.firstname \
                ORDER BY f.name, usage DESC"
    df = pd.read_sql_query(query, conn)
    return df

def facilities_usage_month(conn):
    query = "SELECT f.name AS facility_name, strftime('%Y-%m', b.starttime) AS month, SUM(b.slots) AS usage FROM Bookings b JOIN \
                Facilities f ON b.facid = f.facid WHERE b.memid != 0 GROUP BY f.name, month ORDER BY f.name, month"
    df = pd.read_sql_query(query, conn)
    return df

def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("0. Query all tasks")
        select_all_tasks(conn)
        
        print("\n1. Facilities that charge fees")
        q1_df = facilities_that_charge(conn)
        print(q1_df)

        print("\n2. Facilities that do not charge fees")
        q2_df = facilities_no_charge(conn)
        print(q2_df)

        print("\n3. Facilities whose fees are less than 20% of monthly maintenance")
        q3_df = less_than_maint(conn)
        print(q3_df)

        print("\n4. Facilities with ID 1 and 5")
        q4_df = one_and_five(conn)
        print(q4_df)

        print("\n5. Facilities labelled as cheap or expensive")
        q5_df = cheap_or_expensive(conn)
        print(q5_df)

        print("\n6. Last member/s who joined")
        q6_df = last_members(conn)
        print(q6_df)

        print("\n7. Members who have used a tennis court")
        q7_df = tennis_members(conn)
        print(q7_df)

        print("\n8. Bookings on 2012-09-14 which exceed $30")
        q8_df = more_than_30(conn)
        print(q8_df)

        print("\n9. Bookings on 2012-09-14 which exceed $30 (subquery used)")
        q9_df = more_than_30_subquery(conn)
        print(q9_df)

        print("\n10. Facilities with a total revenue less than 1000")
        q10_df = less_than_thousand(conn)
        print(q10_df)

        print("\n11. Report of members and recommenders")
        q11_df = member_report(conn)
        print(q11_df)

        print("\n12. Facilities usage by member, excluding guests")
        q12_df = facilities_usage_member(conn)
        print(q12_df)

        print("\n13. Facilities usage by month, excluding guests")
        q13_df = facilities_usage_month(conn)
        print(q13_df)
 
if __name__ == '__main__':
    main()

2.6.0
0. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)

1. Facilities that charge fees
             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court

2. Facilities that do not charge fees
   COUNT(name)
0            4

3. Facilities whose fees are less than 20% of monthly maintenance
   facid            name  membercost  monthlymaintenance
0      0  Tennis Court 1         5.0                 200
1      1  Tennis Court 2         5.0                 200
2      4  Massage Room 1         9.9                3000
3      5  Massage Room 2         9.9                3000
4      6    Squash Court         3.5                  80

4.

C:\Users\aless\AppData\Local\Temp\ipykernel_25896\2574233179.py:14: DeprecationWarning: version is deprecated and will be removed in Python 3.14
  print(sqlite3.version)
